# Usando SentiStrength

## Api do sentistrength

In [55]:
import requests 
import re
import json
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

import plotly
import plotly.plotly as py
print(plotly.__version__)  # version >1.9.4 required
import plotly.graph_objs as go
from plotly.graph_objs import Scatter, Layout


3.7.0


In [2]:
def arrumaQuery(query):
    ''' funcao auxiliar para arrumar a query'''
    return query.replace(' ', '+').replace('\n', '+')

In [3]:
def retornaSentimentoComSentiStrength(query):
    '''
    funcao que avalia os sentimentos utilizando 
    o SentiStrength retornando os 
    resultados posivitos e negativos
    '''
    BaseUrl =  "http://sentistrength.wlv.ac.uk"

    DOMAIN = "AutoDetect";
    SUBMIT = "Detect+Sentiment+in+Domain"
    
    # monta URL
    url = BaseUrl+'/results.php?domain='+DOMAIN+'&submit='+SUBMIT+'&text='+arrumaQuery(query)
    
#     print(url)
    #requisita
    r = requests.get(url = url)
    
    # usa regex para capturar nota
    notaExpr = re.compile(r'has positive strength <b>.*</b> and negative strength <b>.*</b>')
    textoFiltrado = re.findall(notaExpr, str(r.content))
    pegaNotaPositiva = str(textoFiltrado).split('<b>')[1].split('</b>')[0]
    pegaNotaNegativa = str(textoFiltrado).split('<b>')[2].split('</b>')[0]
    
    return {'NotaPositiva':int(pegaNotaPositiva), 'NotaNegativa':int(pegaNotaNegativa)}

In [4]:
# retornaSentimentoComSentiStrength('this is a horrible idea')

{'NotaPositiva': 1, 'NotaNegativa': -4}

In [5]:
# pd.DataFrame(columns=['autor', 'data', 'mensagem', 'NotaPos', 'NotaNeg'])
json = None
with open('exemplo.json', 'r') as arq:
    json = arq.read()
#     for commit in json.loads(arq.read()):
# #         print(commit)
#         nota = retornaSentimentoComSentiStrength(commit['mensagem'])
# #         print('autor:', commit['autor'])
# #         print('data:', commit['data'])
# #         print('mensagem:', commit['mensagem'])
# #         print('sentiment', nota)
#         dados.append([
#             commit['autor'],
#             commit['data'],
#             commit['mensagem'],
#             nota['NotaPositiva'],
#             nota['NotaNegativa']            
#         ])

In [13]:
def converteJsonParaDataFrame(json):
    '''
    Recebe um Json represantando um commit com os campos 
    commit['autor'],
    commit['data'],
    commit['mensagem'] e retorna um DataFrame para análise
    '''
    
    dados = []
    for commit in json.loads(json):
        nota = retornaSentimentoComSentiStrength(commit['mensagem'])
        dados.append([
            commit['autor'],
            commit['data'],
            commit['mensagem'],
            nota['NotaPositiva'],
            nota['NotaNegativa']            
        ])
    dados = pd.DataFrame(dados, columns=['autor', 'data', 'mensagem', 'NotaPos', 'NotaNeg'])
    dados.data = pd.to_datetime(dados.data)
    dados = dados.set_index('data')

def plotaGraficosSentimentoPorUsuario(dados):    
    # itera por cada usuário identificado no Json
    for grupo in dados.groupby('autor'):

    url = plotly.offline.plot({
        'data':[
            go.Bar(
                x=grupo[1].index, # assign x as the dataframe column 'x'
                y=grupo[1]['NotaPos'],
                name='Positive Sentiment'
            ),
            go.Bar(
                x=grupo[1].index, # assign x as the dataframe column 'x'
                y=grupo[1]['NotaNeg'],
                name='Negative Sentiment'
            )
        ],
        "layout": Layout(
            title=grupo[0],
            xaxis=dict(title='Commit Data'),
            yaxis=dict(title='Sentiment Polarity'),
            
        )
    }, filename='graphs/linePlot'+grupo[0]+'.html')


In [65]:
# dados.sort_index()[['NotaPos','NotaNeg', 'mensagem', 'autor']].to_csv('saida_Codigo.csv')
# dados.describe().to_csv('analiseSaida.csv')

In [66]:
# dados

In [64]:

    
#     plt.title(grupo[0])

#     plt.ylim((-5, 5))
#     grupo.plot()
